# Initialization

Load ``numpy``, ``pandas`` and ``plotly`` modules.


In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

Load ``fault`` package from ``lib`` directory. It contains ``Fault`` class which acts as a container for the fault data, and functions to compute fault properties


In [2]:
from lib.fault import *

# Load data

Read csv data as a ``DataFrame``


In [3]:
fault_surfaces_points = pd.read_csv("data/fault_surfaces_points.csv")

Group the data by unique ID of faults and create a list of ``Fault`` objects.

In [4]:
fault_surfaces_points_grouped = fault_surfaces_points.groupby('Fault')

faults = [Fault(idx,f[['X','Y','Z']].to_numpy(), 1) for idx, f in fault_surfaces_points_grouped]


## Compute fault parameters

Following fault properties are being computed:
- strike;
- dip;
- minimal depth;
- maximal depth;
- height of the fault;
- length (width) of the fault;
- area of the fault;
- mean curvatures:
    - at the centre of gravity of the fault;
    - averaged across all points;
    - minimal curvature;
    - maximal curvature;

In [5]:
faults = process_faults(faults)
df_faults = generate_table(faults)

In [6]:
df_faults.head()

strike       dip    depth_min    depth_max       height        length  \
0  4.320980  0.578199 -3855.983296 -4242.613502  4706.926988  13898.696435   
1  2.236016  0.490603 -4854.006668 -5004.536237  4034.125551   3828.123437   
2  1.821551  0.465284 -2652.978340 -3617.215649  6351.965159  20684.441929   
3  3.035495  0.706882 -3416.414888 -3503.291378  2229.510838   1960.209198   
4  2.724955  0.740406 -1363.503760 -1384.716577  1252.292091   2265.914173   

           area      curv  curv_mean  curv_min  curv_max    curv_x    curv_y  
0  5.570550e+07 -0.000030  -0.000029 -0.000030 -0.000029 -0.000061  0.000002  
1  8.709254e+06 -0.000013  -0.000013 -0.000014 -0.000013 -0.000021 -0.000006  
2  1.079439e+08 -0.000007  -0.000007 -0.000007 -0.000007 -0.000004 -0.000010  
3  2.456831e+06  0.000066   0.000065  0.000064  0.000066  0.000015  0.000117  
4  1.792160e+06 -0.000033  -0.000033 -0.000033 -0.000033 -0.000045 -0.000021

In [7]:
fig = go.Figure(data=[
    go.Scatter(
        x=df_faults['strike'], 
        y=df_faults['dip'], 
        opacity=0.85,
        mode='markers',
        marker={
            'size': 5.5,
            'color': df_faults['depth_min']
        }

),

],
)

fig.show()

Dip, strike, height, length and curvature computation based on the plane fit of the fault.

For dip, strike, height, length 1st order surface (plane) is used:

\begin{equation}
    z = c_0 + c_1 x + c_2 y
\end{equation}

For curvature calculation 2nd order surface is used:


\begin{equation}
    z = c_0 + c_1 x + c_2 y + c_3 xy + c_4 x^2 + c_5 y^2
\end{equation}


### 1st order plane fit

In [8]:
fault = faults[8]

fig = go.Figure(data=[
    go.Scatter3d(
        x=fault.data[:,0], 
        y=fault.data[:,1], 
        z=fault.data[:,2],
        opacity=0.85,
        mode='markers',
        marker={
            'size': 1.5,
            'color': fault.data[:,2]
        }
),
       go.Mesh3d(
       x=fault.data[:,0], 
       y=fault.data[:,1], 
       z=fault.z_fit,
       alphahull=-1,
       opacity=0.35
    ),

],
    layout = go.Layout(
        width=950,
        height=800,
        scene = dict(
            aspectmode='data',
    )
))

fig.show()

### 2nd order plane fit

In [9]:

fig = go.Figure(data=[
    go.Scatter3d(
        x=fault.data[:,0], 
        y=fault.data[:,1], 
        z=fault.data[:,2],
        opacity=0.85,
        mode='markers',
        marker={
            'size': 1.5,
            'color': fault.data[:,2]
        }
),
       go.Mesh3d(
       x=fault.data[:,0], 
       y=fault.data[:,1], 
       z=fault.z_fit_sq,
       alphahull=-1,
       opacity=0.35
    ),

],
    layout = go.Layout(
        width=950,
        height=800,
        scene = dict(
            aspectmode='data',
    )
))

fig.show()

## Store the result

Store generated pandas dataframe as a CSV file

In [10]:
df_faults.to_csv("results/1-Fault_metrics.csv")